In [93]:
import pandas as pd
import pulp as p
import random

In [94]:
df = pd.read_csv('./test.csv')
df

,university,cost,cost_utility,location,location_utility,rank,rank_utility
0,Univ_A,10000,50,30,90.0,60,70.0
1,Univ_B,8000,60,150,50.0,50,75.0
2,Univ_C,15000,25,50,83.5,5,97.5
3,Univ_D,28000,0,200,30.0,5,97.5
4,Univ_E,12000,40,350,0.0,60,70.0


In [95]:
problem = p.LpProblem(name='University_Selection', sense=p.LpMaximize)
problem

University_Selection:
MAXIMIZE
None
VARIABLES

In [96]:
# Decision Variables
university = p.LpVariable.dicts("", df['university'], cat='Integer', lowBound=0, upBound=random.randint(0, 20))
university

{'Univ_A': _Univ_A,
 'Univ_B': _Univ_B,
 'Univ_C': _Univ_C,
 'Univ_D': _Univ_D,
 'Univ_E': _Univ_E}

In [97]:
costProblem = p.lpSum(university[df['university'][i]] * df['cost_utility'][i] for i in range(df['university'].count()))
costProblem

50*_Univ_A + 60*_Univ_B + 25*_Univ_C + 40*_Univ_E + 0

In [98]:
locationProblem = p.lpSum(university[df['university'][i]] * df['location_utility'][i] for i in range(df['university'].count()))
locationProblem

90.0*_Univ_A + 50.0*_Univ_B + 83.5*_Univ_C + 30.0*_Univ_D + 0.0

In [99]:
rankProblem = p.lpSum(university[df['university'][i]] * df['rank_utility'][i] for i in range(df['university'].count()))
rankProblem

70.0*_Univ_A + 75.0*_Univ_B + 97.5*_Univ_C + 97.5*_Univ_D + 70.0*_Univ_E + 0.0

In [100]:
problem += 0.73 * costProblem + 0.08 * locationProblem + 0.19 * rankProblem
problem

University_Selection:
MAXIMIZE
57.0*_Univ_A + 62.05*_Univ_B + 43.455*_Univ_C + 20.924999999999997*_Univ_D + 42.5*_Univ_E + 0.0
VARIABLES
0 <= _Univ_A <= 20 Integer
0 <= _Univ_B <= 20 Integer
0 <= _Univ_C <= 20 Integer
0 <= _Univ_D <= 20 Integer
0 <= _Univ_E <= 20 Integer

In [101]:
# Constraints
problem += p.lpSum(university[df['university'][i]] for i in range(df['university'].count())) == 35
problem

University_Selection:
MAXIMIZE
57.0*_Univ_A + 62.05*_Univ_B + 43.455*_Univ_C + 20.924999999999997*_Univ_D + 42.5*_Univ_E + 0.0
SUBJECT TO
_C1: _Univ_A + _Univ_B + _Univ_C + _Univ_D + _Univ_E = 35

VARIABLES
0 <= _Univ_A <= 20 Integer
0 <= _Univ_B <= 20 Integer
0 <= _Univ_C <= 20 Integer
0 <= _Univ_D <= 20 Integer
0 <= _Univ_E <= 20 Integer

In [102]:
# Solve problem
problem.solve()

# Print status
status = p.LpStatus[problem.status]
print("Status:", status)

# Print optimal values of decision variables

university = []
selectedUniversity = ""
for v in problem.variables():
    print(v.name, "=", v.varValue)
    university.append(v.name.replace("_", "") + " = " + str(v.varValue))
    if v.varValue is not None and v.varValue > 0:
        selectedUniversity = v.name
totalUtility = p.value(problem.objective)
print("Total Utility : ", totalUtility)

universitySelected = "Optimal value for supplier's " + "'" + str(university) + "' and the optimized objective function value is " + str(totalUtility)
print(universitySelected)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/t4/204xh5g57vv3c7mjndmh4y7r0000gn/T/c98755aea9554bbc86de01f5803e55fa-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/t4/204xh5g57vv3c7mjndmh4y7r0000gn/T/c98755aea9554bbc86de01f5803e55fa-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 27 RHS
At line 29 BOUNDS
At line 35 ENDATA
Problem MODEL has 1 rows, 5 columns and 5 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2096 - 0.00 seconds
Cgl0004I processed model has 1 rows, 5 columns (5 integer (0 of which binary)) and 5 elements
Cutoff increment increased from 1e-05 to 0.004995
Cbc0012I Integer solution of -2096 found by DiveCoefficient after 0 iterations and 0 nodes